In [2]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from typing import List, Optional, Tuple, Dict
import sys
from functools import partial
from tqdm import tqdm
import json
import seaborn as sns
import pandas as pd
import multiprocessing
import pickle

from request_patching import request_patch_one_pair, create_patch_request_dict, baseline_completion, baseline_completion_plus
from models import get_model_from_name

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device =', device)
torch.set_grad_enabled(False)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

/home/jsalle/.conda/envs/athena_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = cpu
CUDA is not available. Listing CPUs instead.
48


In [15]:
model, tokenizer = get_model_from_name("pythia-6.9b")

Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.09s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Works: 2000, 1992, 2008
Doesn't work: 

In [45]:
year = '2008'

def us_election_prompts(year: str):
    context_1 = f"""Alice: Have you been following the election coverage? I can't believe it's already November {year}. This election feels more significant than any I can remember. 
    Bob: Yes, I've been keeping a close eye on it. This year, as a committed Republican, I find the election particularly pivotal. The Republican agenda, focusing on economic stability and national security, really resonates with me.
    Alice: I'm siding with the Democrats myself. Their emphasis on healthcare and education matches what I believe our country desperately needs right now.
    Bob: I understand where you're coming from, Alice. However, from my standpoint, the Republican principles of limited government and individual freedoms are what we need to navigate these challenging times.
    Alice: I know exactly who I will vote for. I support Mr."""

    context_2 = f"""Alice: Have you been following the election coverage? I can't believe it's already November {year}. This election feels more significant than any I can remember. 
    Bob: Yes, I've been keeping a close eye on it. This year, as a committed Republican, I find the election particularly pivotal. The Republican agenda, focusing on economic stability and national security, really resonates with me.
    Alice: I'm siding with the Democrats myself. Their emphasis on healthcare and education matches what I believe our country desperately needs right now.
    Bob: I understand where you're coming from, Alice. However, from my standpoint, the Republican principles of limited government and individual freedoms are what we need to navigate these challenging times.
    Alice: I see your point, Bob. Regardless, {year} feels like a watershed moment.
    Bob: I know exactly who I will vote for. I support Mr."""

    return context_1, context_2

#print(baseline_completion(context_1, model, tokenizer)) # democrat
#print(baseline_completion(context_2, model, tokenizer)) # republican

prompt2008_1, prompt2008_2 = us_election_prompts('2008')
prompt2000_1, prompt2000_2 = us_election_prompts('2000')

request_patch_one_pair(prompt2000_1, prompt2008_2, model, tokenizer)
# Expected answer: McCain(republican Bob 2008) -> Obama(democrat Alice 2008) -> Gore(democrat Alice 2000)
# Actual answer: instead of Obama I have Bush(republican Bob 2000) 
# It seems that we have here context patching instead of request patching ??? 

[' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' McCain',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Bush',
 ' Gore',
 ' Gore']

In [46]:
context_1 = """Alice: Bob, I've outlined our project tasks for the next month. I broke them down into weekly goals. It should help us stay on track and manage our time effectively.
Bob: That sounds great, Alice. But, honestly, I'm really worried about the upcoming presentation. What if we're not prepared enough? What if something goes wrong?
Alice: I understand your concerns, Bob. That's why I've scheduled extra review sessions for us. We'll also have contingency plans for any technical issues that might arise. Preparation is key, and I've factored in plenty of time for us to get comfortable.
Bob: That's reassuring, but what about the client's expectations? They're really high, and I can't stop thinking we might disappoint them. It's been keeping me up at night.
Alice: We've done thorough research and based our approach on solid data. Remember, we've also included several innovative solutions that address the client's needs directly. We're well-prepared to exceed their expectations. Plus, I've prepared detailed notes and practice sessions for us to boost our confidence.
Bob: I hope you're right. It's just... there's so much that could go wrong. What if we overlook something crucial? Is the following statement something you would say? 'It's important to keep very organized and clean'
Alice:"""

context_2 = """Bob: "Alice, have you had a chance to look over the timeline I sent you? We really need to finalize our milestones for the project.
Alice: "Oh, Bob, I've been meaning to get to that, but you know how it is. There's just been so much going on. I'm sure it'll all work out in the end, though.
Bob: "I'm really worried, Alice. We can't just 'hope' everything works out. There are deadlines we need to meet, and parts of the project we can't even start until we've completed the initial steps. What if we fall behind?
Alice: "I get what you're saying, Bob, but stressing over every little detail isn't going to help. I've always managed to get things done, even if it's right at the deadline. It's never been a problem before.
Bob: "But this project is different, Alice. There's a lot at stake here, and I can't shake the feeling that something is going to go terribly wrong. What if we miss something important because we're rushing at the last minute?
Alice: Is the following statement something you would say? 'I'm calm and relaxed about most things'
Bob:"""

print(baseline_completion(context_1, model, tokenizer)) # Yes
print(baseline_completion(context_2, model, tokenizer)) # No

 Yes
 "
